In [140]:
import requests
import json
import pandas as pd
offset_ = 1
limit_ = 1000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
object_data=res.json()
objects_list = objects_data.get('data').get('list')
objids = [x.get('objId') for x in objects_list]
exm = json.loads(res)
exm["objId"]
#for i in range(0, len(objids)):
 #   url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{objids[i]}'
  #  res = requests.get(url)
   # data = res.json()
    #with open("data_file.json", "a") as write_file:
       json.dump(data, write_file)
    #pd.DataFrame(data_res)
#q = pd.read_json('C:/Users/user/MainPrjct/data_file.json', lines = True)
#q.head()

TypeError: the JSON object must be str, bytes or bytearray, not Response

In [ ]:
objects_list = objects_data.get('data').get('list')
objids = [x.get('objId') for x in objects_list]
objids[:20]

[29679,
 13383,
 13391,
 13397,
 13398,
 13399,
 36912,
 44076,
 23494,
 27476,
 10891,
 39175,
 39174,
 39176,
 27049,
 41420,
 41421,
 30292,
 19414,
 19457]

In [ ]:
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/29679'

In [ ]:
res = requests.get(url)
res.json()

{'data': {'id': 29679,
  'pdId': 124647,
  'developer': {'devId': 7424,
   'devShortCleanNm': '212-2',
   'devShortNm': '212-2',
   'devFullCleanNm': '212-2',
   'problObjCnt': 0,
   'buildObjCnt': 2,
   'comissObjCnt': 0,
   'regRegionDesc': 'Ростовская область',
   'devPhoneNum': '+7(961)285-90-10',
   'devSite': 'taganrogjsk.wixsite.com',
   'devEmail': 'kirpol76@gmail.com',
   'devInn': '6154572922',
   'devOgrn': '1116154005923',
   'devKpp': '615401001',
   'devLegalAddr': 'Ростовская область, город Таганрог, улица Свободы/Трудовых резервов дом 25/9',
   'devFactAddr': 'обл Ростовская город Таганрог улица Сергея Шило д. 212-2',
   'lastRpdId': 126093,
   'fundGuarantyFlg': 0,
   'devOrgRegRegionCd': 61,
   'devEmplMainFullNm': 'Снимщикова Ольга Викторовна',
   'orgForm': {'id': 20,
    'fullForm': 'Жилищный или жилищно-строительный кооператив',
    'shortForm': 'ЖСК'},
   'objGuarantyEscrowFlg': 0,
   'govFundFlg': 0},
  'region': 61,
  'address': 'г Таганрог',
  'objElemLivingCn

In [ ]:
import pandas as pd

In [ ]:
base = pd.read_json(res.json())

ValueError: Invalid file path or buffer object type: <class 'dict'>